 _a python code which takes mic recording as an input and adds delay to it as an output_ 
 -------------------------
 Explanation:

First, we import the required modules and define some constants like chunk size, audio format, channels, sample rate, and delay.

Next, we initialize a PyAudio object and open an audio stream for recording and playback.

We start recording audio from the input stream and store the audio frames in a list for later playback.

After the recording is complete, we iterate through the recorded audio frames and play them back through the output stream.

Finally, we close the audio stream and terminate the PyAudio object.

In [6]:
import pyaudio
import numpy as np
import pywt
import wave

# Constants
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK_SIZE = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "cleaned_audio.wav"

# Set up PyAudio object
p = pyaudio.PyAudio()

# Open microphone input stream
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK_SIZE)

print("Recording...")

# Record audio for specified number of seconds
frames = []
for i in range(0, int(RATE / CHUNK_SIZE * RECORD_SECONDS)):
    data = stream.read(CHUNK_SIZE)
    frames.append(data)

print("Finished recording")

# Convert frames to numpy array
audio_data = np.frombuffer(b''.join(frames), dtype=np.int16)

# Apply Discrete Wavelet Transform and Soft Thresholding for noise reduction
wavelet_name = 'db4'
level = 3
threshold = 0.02  # adjust as needed
coeffs = pywt.wavedec(audio_data, wavelet_name, level=level)
new_coeffs = [pywt.threshold(c, threshold*max(c)) for c in coeffs]
new_audio = pywt.waverec(new_coeffs, wavelet_name)

# Convert audio data back to bytes
new_frames = new_audio.astype(np.int16).tobytes()

# Save cleaned audio to WAV file
with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(new_frames)

# Clean up
stream.stop_stream()
stream.close()
p.terminate()

print("Saved cleaned audio to " + WAVE_OUTPUT_FILENAME)

Recording...
Finished recording
Saved cleaned audio to cleaned_audio.wav
